<a href="https://colab.research.google.com/github/reemchaaban/game_system/blob/main/data-processing/game_rec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAME RECOMMENDATION MODEL

Installing needed packages

In [1]:
!pip install lightfm --quiet
!pip install huggingface_hub --quiet
!pip install mlflow --quiet
!pip install pyngrok --quiet
!pip install prometheus-client --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00


Importing needed libraries

In [2]:
import os
import json
from google.colab import drive, files, userdata
import shutil
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import pickle
from scipy.sparse import coo_matrix

from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k, reciprocal_rank, auc_score
from sklearn.preprocessing import MinMaxScaler

from huggingface_hub import login, upload_folder
from pyngrok import ngrok
import mlflow
from tqdm.notebook import trange
from tqdm import trange

from prometheus_client import start_http_server, Gauge, REGISTRY
import psutil
import threading
import time

Authenticating Hugging Face & ngrok using tokens from Secrets

In [3]:
HF_TOKEN = userdata.get("HF_TOKEN")
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

login(token=HF_TOKEN)

Loading dataset from Drive

In [4]:
drive.mount('/content/drive')
drive_base_path = '/content/drive/My Drive/503Nproj/datasets'

games_df = pd.read_csv(os.path.join(drive_base_path, 'game_library_data.csv'))

file_path = os.path.join(drive_base_path, 'synthetic_training_data.json')

with open(file_path, 'r') as file:
    player_data = json.load(file)

Mounted at /content/drive


Flatten nested JSON player libraries into dataframe

In [5]:
records = []
for player_id, pdata in player_data.items():
    for game_id, gdata in pdata['games'].items():
        records.append({
            'player_id': player_id,
            'game_id': game_id,
            'game_name': gdata['name'],
            'rating_ratio': float(gdata['rating_ratio']),
            'price': float(gdata['price']),
            'genres': gdata['genre'], #list
            'tags': gdata['tags'], #list
            'playtime_hours': gdata['hours']
        })

player_libraries = pd.DataFrame(records)

#normalizing playtime per player, where most-played game by player = 1.0 & others are relative
player_libraries['normalized_playtime'] = player_libraries.groupby('player_id')['playtime_hours'].transform(lambda x: x / x.max())

In [6]:
player_libraries.head()

,player_id,game_id,game_name,rating_ratio,price,genres,tags,playtime_hours,normalized_playtime
0,0,1551360,Forza Horizon 5,7.785,5999.0,"[Action, Adventure, Racing, Simulation, Sports]","[Racing, Open World, Driving, Multiplayer, Aut...",49,0.156550
1,0,244210,Assetto Corsa,12.532,1999.0,"[Indie, Racing, Simulation, Sports]","[Racing, Automobile Sim, Simulation, Driving, ...",235,0.750799
2,0,284160,BeamNG.drive,37.755,2499.0,"[Racing, Simulation, Early Access]","[Simulation, Driving, Physics, Realistic, Dest...",313,1.000000
3,0,252950,Rocket League,6.945,0.0,"[Action, Indie, Racing, Sports]","[Multiplayer, Football (Soccer), Competitive, ...",92,0.293930
4,0,211500,RaceRoom Racing Experience,3.045,0.0,"[Racing, Simulation, Sports, Free To Play]","[Free to Play, Racing, Automobile Sim, Realist...",3,0.009585


Building dataset

In [8]:
dataset = Dataset()
dataset.fit(
    users=player_libraries['player_id'],
    items=games_df['game_id']
)

Normalization & saving game-rec scaler

In [9]:
#normalizing price & rating ratio
scaler = MinMaxScaler()
player_libraries[['norm_price', 'norm_rating_ratio']] = scaler.fit_transform(player_libraries[['price', 'rating_ratio']])

#saving scaler
os.makedirs("game-rec", exist_ok=True)
with open("game-rec/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

Creating item-feature matrix

In [10]:
#combining features
player_libraries['feature_list'] = player_libraries.apply(
    lambda row: [f'genre:{g}' for g in row['genres']] + #multi-hot categorical feature
                [f'tag:{t}' for t in row['tags']] + #multi-hot categorical feature
                [f'norm_price:{row.norm_price:.4f}'] +
                [f'norm_rating_ratio:{row.norm_rating_ratio:.4f}'], axis=1)

#grouping by game ID & aggregate features, i.e., genre, tag, price, rating ratio
item_features_df = player_libraries.groupby('game_id').agg({
    'feature_list': lambda lists: lists.iloc[0]  #features are the same for each game
}).reset_index()

In [11]:
#fill with all item features
all_features = set()
for feats in item_features_df['feature_list']:
    all_features.update(feats)

dataset.fit_partial(
    items=item_features_df['game_id'],
    item_features=list(all_features)
)

In [12]:
item_features_tuples = list(zip(item_features_df['game_id'], item_features_df['feature_list']))
item_features_matrix = dataset.build_item_features(item_features_tuples)

with open("game-rec/item_features_matrix.pkl", "wb") as f:
    pickle.dump(item_features_matrix, f)

In [13]:
#building interaction matrix for LightFM w/ weights
(interactions, weights) = dataset.build_interactions([
    (row['player_id'], row['game_id'], row['normalized_playtime'])
    for _, row in player_libraries.iterrows()
])

Prometheus server

In [15]:
for collector in list(REGISTRY._collector_to_names.keys()):
    REGISTRY.unregister(collector)

def start_prometheus_server():
    start_http_server(8000)
    print("Prometheus server started on http://localhost:8000/")

threading.Thread(target=start_prometheus_server).start()

Prometheus server started on http://localhost:8000/

In [17]:
# terminate any open tunnels
ngrok.kill()

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
prometheus_url = ngrok.connect(8000, proto="http")
print(f" * Prometheus tunnel: {prometheus_url}")

 * Prometheus tunnel: NgrokTunnel: "https://ad7e-34-81-191-127.ngrok-free.app" -> "http://localhost:8000"


Defining Prometheus metrics

In [18]:
cpu_usage_metric = Gauge('game_model_cpu_usage_percent', 'CPU usage (%)')
memory_usage_metric = Gauge('game_model_memory_usage_percent', 'Memory usage (%)')
epoch_duration_metric = Gauge('game_model_epoch_duration_seconds', 'Epoch duration')

LightFM model training & evaluation

In [19]:
model = LightFM(loss='warp')

num_epochs = 60
num_threads = 4

with mlflow.start_run(run_name="game-recommender"):
    mlflow.log_param("loss", "warp")
    mlflow.log_param("components", 30)

    for epoch in trange(num_epochs, desc="Training epochs"):
        start_time = time.time()

        model.fit_partial(
            interactions,
            sample_weight=weights,
            item_features=item_features_matrix,
            epochs=1,
            num_threads=num_threads
        )

        # evaluation
        precision = precision_at_k(model, interactions, k=5, item_features=item_features_matrix).mean()
        recall = recall_at_k(model, interactions, k=5, item_features=item_features_matrix).mean()
        auc = auc_score(model, interactions, item_features=item_features_matrix).mean()

        # logging metrics to MLflow
        mlflow.log_metric("Precision", precision, step=epoch)
        mlflow.log_metric("Recall", recall, step=epoch)
        mlflow.log_metric("AUC", auc, step=epoch)

        # logging system metrics to Prometheus
        duration = time.time() - start_time
        cpu = psutil.cpu_percent()
        mem = psutil.virtual_memory().percent

        epoch_duration_metric.set(duration)
        cpu_usage_metric.set(cpu)
        memory_usage_metric.set(mem)

        print(f"Epoch {epoch}: Duration {duration:.2f}s | CPU {cpu}% | Memory {mem}%")

Training epochs:   2%|▏         | 1/60 [00:18<18:03, 18.36s/it]

Epoch 0: Duration 18.36s | CPU 38.4% | Memory 42.9%


Training epochs:   3%|▎         | 2/60 [00:36<17:28, 18.08s/it]

Epoch 1: Duration 17.88s | CPU 74.4% | Memory 42.8%


Training epochs:   5%|▌         | 3/60 [00:55<17:42, 18.64s/it]

...

Epoch 57: Duration 17.47s | CPU 74.0% | Memory 42.9%


Training epochs:  98%|█████████▊| 59/60 [17:53<00:17, 17.81s/it]

Epoch 58: Duration 17.39s | CPU 73.5% | Memory 42.7%


Training epochs: 100%|██████████| 60/60 [18:10<00:00, 18.18s/it]

Epoch 59: Duration 17.52s | CPU 73.7% | Memory 42.8%


Model evaluation

In [20]:
# precision at k = how many of top-K recommended items were relevant
precision = precision_at_k(model, interactions, item_features=item_features_matrix, k=5).mean()
print(f'Precision at k=5: {precision:.4f}')

# recall at k = how many of top-k recommended items were relevant & successfully recommended
recall = recall_at_k(model, interactions, item_features=item_features_matrix, k=5).mean()
print(f'Recall at k=5: {recall:.4f}')

# mean reciprocal rank = measures rank position of 1st relevant item in rec list
mrr = reciprocal_rank(model, interactions, item_features=item_features_matrix).mean()
print(f'MRR: {recall:.4f}')

Precision at k=5: 0.4310
Recall at k=5: 0.1408
MRR: 0.1408


Launch MLflow UI locally on port 5000

In [21]:
! pkill -f "mlflow ui"

get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background

Expose using ngrok

In [22]:
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

mlflow_url = ngrok.connect(5000, proto="http")

print(f" * Ngrok tunnel: {mlflow_url}")

 * Ngrok tunnel: NgrokTunnel: "https://6a00-34-81-191-127.ngrok-free.app" -> "http://localhost:5000"


Generate recommendations

In [23]:
#sample player ID
sample_player_id = player_libraries['player_id'].iloc[98375]

all_game_ids = games_df['game_id'].tolist()

#map player/game IDs to LightFM indices
user_id_map, _, item_id_map, item_feature_map = dataset.mapping()
with open("game-rec/dataset_mappings.pkl", "wb") as f:
    pickle.dump({
        "user_id_map": user_id_map,
        "item_id_map": item_id_map,
        "item_feature_map": item_feature_map
    }, f)
user_index = user_id_map[sample_player_id]

#predict scores for all games
scores = model.predict(
    user_ids=user_index,
    item_ids=np.arange(len(item_id_map)),
    item_features=item_features_matrix
)

#get top N game indices (that are not in the player's library)
played_game_ids = player_libraries[player_libraries['player_id'] == sample_player_id]['game_id'].tolist()
played_game_indices = [item_id_map[g] for g in played_game_ids if g in item_id_map]

#mask out games in player's library already
scores[played_game_indices] = -np.inf

#top 5 recommendations
top_indices = np.argsort(-scores)[:5]

#reverse mapping of item_index to game_id
inv_item_id_map = {v: k for k, v in item_id_map.items()}
recommended_game_ids = [inv_item_id_map[i] for i in top_indices]
recommended_game_ids = [int(gid) for gid in recommended_game_ids]

#print names of recommended games
print("Recommended games for player with ID", sample_player_id)
for gid in recommended_game_ids:
    match = games_df[games_df['game_id'] == gid]
    if not match.empty:
        name = match['name'].values[0]
    else:
        name = f"(Game ID {gid} not found)"
    print(f" - {name}")

Recommended games for player with ID 6145
 - Business Tour - Board Game with Online Multiplayer
 - BattleBlock Theater
 - Sven Co-op
 - Paladins
 - Garry's Mod


Save trained model

In [ ]:
with open('game-rec.pkl', 'wb') as f:
    pickle.dump(model, f)

shutil.copy("game-rec.pkl", "game-rec/game-rec.pkl")

'game-rec/game-rec.pkl'

In [ ]:
upload_folder(
    repo_id="reemchaaban/game-rec",
    folder_path="game-rec",
    path_in_repo=".",
    repo_type="model"
)

dataset_mappings.pkl:   0%|          | 0.00/558k [00:00<?, ?B/s]

game-rec.pkl:   0%|          | 0.00/6.73M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/reemchaaban/game-rec/commit/42ace7e6bc7b93604b69af3af28afb6a607e85e8', commit_message='Upload folder using huggingface_hub', commit_description='', oid='42ace7e6bc7b93604b69af3af28afb6a607e85e8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/reemchaaban/game-rec', endpoint='https://huggingface.co', repo_type='model', repo_id='reemchaaban/game-rec'), pr_revision=None, pr_num=None)

Automating update of model & scaler files in IEP2 folder after retraining

In [ ]:
GITHUB_USERNAME = "reemchaaban"
GITHUB_EMAIL = "reem.chaabann@gmail.com"
REPO_NAME = "game_system"
BRANCH = "main"
TARGET_SUBDIR = "IEP2"

token = userdata.get('GITHUB_PAT')

!git config --global user.email "{GITHUB_EMAIL}"
!git config --global user.name "{GITHUB_USERNAME}"

!rm -rf {REPO_NAME}
!git clone https://{token}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git

!rm -rf {REPO_NAME}/{TARGET_SUBDIR}/model
!mkdir -p {REPO_NAME}/{TARGET_SUBDIR}/model
!cp game-rec/dataset_mappings.pkl {REPO_NAME}/{TARGET_SUBDIR}/model/
!cp game-rec/item_features_matrix.pkl {REPO_NAME}/{TARGET_SUBDIR}/model/

!cp game-rec.pkl {REPO_NAME}/{TARGET_SUBDIR}/model/game-rec.pkl

%cd {REPO_NAME}
!git add .
!git commit -m "Update IEP2 model and scaler from Colab"
!git push origin {BRANCH}

%cd ..

Cloning into 'game_system'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 161 (delta 20), reused 16 (delta 5), pack-reused 114 (from 1)
Receiving objects: 100% (161/161), 16.95 MiB | 26.45 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/game_system
[main b6b5bd8] Update IEP2 model and scaler from Colab
 3 files changed, 0 insertions(+), 0 deletions(-)
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 2 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 3.89 MiB | 6.07 MiB/s, done.
Total 7 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/reemchaaban/game_system.git
   3afe87e..b6b5bd8  main -> main
/content
